In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# 1. 读取数据
df = pd.read_excel('tree.xlsx', sheet_name='Sheet1')

# 2. 文本预处理函数：只保留 "ind_xxx" 这样的特征名
# 例如将 "ind_4e < 1 or missing" 转化为 "ind_4e"
def preprocess_rule(text):
    import re
    # 正则表达式匹配以 ind 开头的特征名
    features = re.findall(r"(ind[\w_]+)", str(text))
    return " ".join(features)

# 3. 应用预处理
df['features_text'] = df['DetailedSplit'].apply(preprocess_rule)

# 4. 构建矩阵 (One-Hot Encoding)
# binary=True 表示我们只关心“有没有用到这个特征”，不关心用了几次
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(df['features_text'])

# 5. 转化为 DataFrame 查看 (这就是“规则-特征”矩阵)
rule_feature_matrix = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# 将原始的 Point（分数）拼回来，方便对照
rule_feature_matrix['Score_Points'] = df['Point']

print(rule_feature_matrix.head())

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# ==========================================
# 步骤 0: 准备工作
# ==========================================
# 假设这是你所有的 17 个指标名称（请把这里换成你真实的列表）
# 即使某些指标在 rules 里一次都没出现，它们也会作为全 0 列出现在结果中
all_17_indicators = [
    "ind_4e", "ind_13b", "ind_3a_1", "ind_12f", "ind_2a_1", 
    "ind_13a_1", "ind_6", "ind_7", "ind_8", "ind_9", 
    "ind_10", "ind_11", "ind_14", "ind_15", "ind_16", 
    "ind_17", "ind_unused_example" # 确保这里列出了全部 17 个
]

# 读取数据
df = pd.read_excel('tree.xlsx', sheet_name='Sheet1')

# ==========================================
# 步骤 1: 文本预处理 (保持 210 行不变)
# ==========================================
# 我们只提取特征名，不拆分行
def extract_features(text):
    import re
    if pd.isna(text): return ""
    # 提取所有 ind_ 开头的词
    feats = re.findall(r"(ind[\w_]+)", str(text))
    return " ".join(feats)

df['feature_text'] = df['DetailedSplit'].apply(extract_features)

# ==========================================
# 步骤 2: 构建矩阵 (强制使用所有 17 个指标)
# ==========================================
# 关键点：使用 vocabulary 参数！
# 这告诉程序：“只关注这17个词，其他的我不要；没出现的词也要给我留列位置。”
vectorizer = CountVectorizer(binary=True, vocabulary=all_17_indicators)

# 生成矩阵
X = vectorizer.fit_transform(df['feature_text'])

# 转化为 DataFrame，列名就是我们指定的顺序
matrix_df = pd.DataFrame(X.toarray(), columns=all_17_indicators)

# ==========================================
# 步骤 3: 拼接结果 (加上 Rule 和 Point)
# ==========================================
# axis=1 表示左右横向拼接
final_df = pd.concat([
    df[['DetailedSplit', 'Point']],  # 第一列显示 Rule，第二列显示分数
    matrix_df                        # 后面跟着 17 列指标矩阵
], axis=1)

# ==========================================
# 步骤 4: 检查与保存
# ==========================================
print(f"最终矩阵维度: {final_df.shape}") 
# 预期输出: (210, 2 + 17) = (210, 19)

# 预览一下
print(final_df.head())

# 保存结果
final_df.to_excel("final_rule_matrix.xlsx", index=False)